# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model

In [57]:
!wget -O train.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/test.txt

--2022-10-10 22:37:51--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11185077 (11M) [text/plain]
Saving to: 'train.txt'

     0K .......... .......... .......... .......... ..........  0% 1.39M 8s
    50K .......... .......... .......... .......... ..........  0% 58.2M 4s
   100K .......... .......... .......... .......... ..........  1% 27.9M 3s
   150K .......... .......... .......... .......... ..........  1% 9.45M 2s
   200K .......... .......... .......... .......... ..........  2% 11.1M 2s
   250K .......... .......... .......... .......... ..........  2% 7.73M 2s
   300K .......... .......... .......... .......... ..........  3% 12.7M 2s
   350K ......

--2022-10-10 22:37:54--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1385157 (1.3M) [text/plain]
Saving to: 'dev.txt'

     0K .......... .......... .......... .......... ..........  3% 5.13M 0s
    50K .......... .......... .......... .......... ..........  7% 18.1M 0s
   100K .......... .......... .......... .......... .......... 11% 14.2M 0s
   150K .......... .......... .......... .......... .......... 14% 9.34M 0s
   200K .......... .......... .......... .......... .......... 18% 16.3M 0s
   250K .......... .......... .......... .......... .......... 22% 12.5M 0s
   300K .......... .......... .......... .......... .......... 25% 19.5M 0s
   350K ..........

### 1.1 Building vocabulary

#### Code

In [65]:
import nltk

SOS = "<s>"
EOS = "</s>"
UNK = "<UNK>"


def load_data(path):
    with open(path, encoding='utf-8') as f:
        data = [l.strip() for l in f.readlines()]
    return data


def add_sentence_tokens(sentences):
    return ['{} {} {}'.format(SOS, s, EOS) for s in sentences]


def replace_oov(tokens):
    vocab = nltk.FreqDist(tokens)
    return [token if vocab[token] >= 3 else UNK for token in tokens]


def pre_process(sentences):
    sentences = add_sentence_tokens(sentences)
    tokens = ' '.join(sentences).split(' ')
    tokens = [token.lower() for token in tokens]
    tokens = replace_oov(tokens)
    return tokens


train = load_data("train.txt")
dev = load_data("dev.txt")
test = load_data("test.txt")
tokens = pre_process(train)
vocab = nltk.FreqDist(tokens)
vocab_size = len(vocab)
print("Vocabulary size: {}".format(vocab_size))

Vocabulary size: 20663


#### Discussion

 Vocabulary size: 20663<br/>
 the number of parameters of n-gram model:$$20663^n$$

### 1.2 $n$-gram Language Modeling

#### Code

In [59]:
from itertools import product
import math


def build_multi_gram(n):
    n_grams = nltk.ngrams(tokens, n)
    n_vocab = nltk.FreqDist(n_grams)
    if n < 2:
        return {n_gram: count/len(tokens) for n_gram, count in n_vocab.items()}
    m_grams = nltk.ngrams(tokens, n - 1)
    m_vocab = nltk.FreqDist(m_grams)
    return {n_gram: count/m_vocab[n_gram[:-1]] for n_gram, count in n_vocab.items()}


def convert_oov(model, ngram, n):
    masks = list(reversed(list(product((0, 1), repeat=n))))
    mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))
    ngram = (ngram,) if type(ngram) is str else ngram
    for possible_known in [mask(ngram, bitmask) for bitmask in masks]:
        if possible_known in model:
            return possible_known


def cal_perplexity(test, n, model):
    test_tokens = pre_process(test)
    test_ngrams = nltk.ngrams(test_tokens, n)
    M = len(test_tokens)
    known_ngrams = (convert_oov(model, ngram, n) for ngram in test_ngrams)
    probs = [model[ngram] for ngram in known_ngrams]
    perplex = pow(2, (-1/M)*sum(map(math.log, probs)))
    return perplex


unigram_m = build_multi_gram(1)
bigram_m = build_multi_gram(2)

perplex_train_1 = cal_perplexity(train, 1, unigram_m)
perplex_dev_1 = cal_perplexity(dev, 1, unigram_m)
perplex_train_2 = cal_perplexity(train, 2, bigram_m)
perplex_dev_2 = cal_perplexity(dev, 2, bigram_m)
print("Perplexity of unigram model on training set: {}\n"
      "Perplexity of unigram model on dev set: {}\n"
      "Perplexity of bigram model on training set: {}\n"
      "Perplexity of bigram model on dev set: {}".format(perplex_train_1, perplex_dev_1, perplex_train_2,
                                                               perplex_dev_2))

Perplexity of unigram model on training set: 93.99585636227933
Perplexity of unigram model on dev set: 75.43207223220776
Perplexity of bigram model on training set: 17.29268572614725
Perplexity of bigram model on dev set: 16.78284067451732


#### Discussion

Perplexity of unigram model on training set: 93.99585636227933<br/>
Perplexity of unigram model on dev set: 75.43207223220776<br/>
Perplexity of bigram model on training set: 17.29268572614725<br/>
Perplexity of bigram model on dev set: 16.78284067451732<br/>

Perplexity of unigram model on training set is larger than that on dev set, because UNK is used to replace words in test set which are not contained in training vocabulory, and the probility of UNK is large.<br/>
Perplexity of bigram model on training set is similar with that on dev set, because even some words are replaced by UNK, P(Xi,UNK) is not that large compared with P(UNK) in unigram model.<br/>
Perplexity of unigram model is larger than that of bigram model, because bigram model has context information and it can predict more precisely.

### 1.3 Smoothing

#### 1.3.1 Add-one (Laplace) smoothing

##### Code

In [60]:
def laplace(k):
    grams_2 = nltk.ngrams(tokens, 2)
    vocab_2 = nltk.FreqDist(grams_2)

    grams_1 = nltk.ngrams(tokens, 1)
    vocab_1 = nltk.FreqDist(grams_1)
    return {gram_2: (count + k) / (vocab_1[gram_2[:-1]] + vocab_size * k) for gram_2, count in vocab_2.items()}

bigram_m = laplace(1)

perplex_train = cal_perplexity(train, 2, bigram_m)
perplex_dev = cal_perplexity(dev, 2, bigram_m)
print("Perplexity of bigram model on training set after smoothing: {}\n"
      "Perplexity of bigram model on dev set after smoothing: {}".format(perplex_train, perplex_dev))

Perplexity of bigram model on training set after smoothing: 86.515018275416
Perplexity of bigram model on dev set after smoothing: 63.2280700395138


##### Discussion

Perplexity of bigram model on training set after smoothing: 86.515018275416<br/>
Perplexity of bigram model on dev set after smoothing: 63.2280700395138<br/>

Perplexity on training set and dev set both increase, because according to:
$$P(Ci-1|Ci)=(P(Ci-1,Ci)+1)/(P(Ci-1)+v)$$
the denominator of the probability increases vocab_size times faster than the numerator.

##### Optional: Add-k smoothing

###### Code

In [61]:
list_k = [0.01, 0.05, 0.5]
for k in list_k:
    bigram_m = laplace(k)
    perplex_train = cal_perplexity(train, 2, bigram_m)
    perplex_dev = cal_perplexity(dev, 2, bigram_m)
    print("When k={}:\n"
          "Perplexity of bigram model on training set: {}\n"
          "Perplexity of bigram model on dev set: {}".format(k, perplex_train, perplex_dev))

When k=0.01:
Perplexity of bigram model on training set: 23.570739594539667
Perplexity of bigram model on dev set: 20.171703988957148
When k=0.05:
Perplexity of bigram model on training set: 32.73908329303373
Perplexity of bigram model on dev set: 26.07385239496752
When k=0.5:
Perplexity of bigram model on training set: 67.07190203797246
Perplexity of bigram model on dev set: 49.29044098488054


###### Discussion

When k=0.01:<br/>
Perplexity of bigram model on training set: 23.570739594539667<br/>
Perplexity of bigram model on dev set: 20.171703988957148<br/>
When k=0.05:<br/>
Perplexity of bigram model on training set: 32.73908329303373<br/>
Perplexity of bigram model on dev set: 26.07385239496752<br/>
When k=0.5:<br/>
Perplexity of bigram model on training set: 67.07190203797246<br/>
Perplexity of bigram model on dev set: 49.29044098488054<br/>

Larger k leading to larger perplexity, because according to:
$$P(Ci-1|Ci)=(P(Ci-1,Ci)+k)/(P(Ci-1)+k*v)$$
when k increase, the denominator of the probability linearly increases v times. Larger k leads to smaller probility of sentences in train sets.

#### 1.3.2 Linear Interpolation

##### Code

In [62]:
def linear_interpolation(k_list, vocab_l):
    def p_3(gram_3, count):
        n_2 = vocab_l[1][gram_3[:-1]]
        if n_2 == 0:
            return 0
        return count/n_2

    def p_2(gram_3):
        n_1 = vocab_l[0][gram_3[1:-1]]
        if n_1 == 0:
            return 0
        return vocab_l[1][gram_3[1:]]/n_1

    def p_1(gram_3):
        t = vocab_l[0][gram_3[2:]] / len(tokens)
        return t

    return {gram_3: k_list[0] * p_3(gram_3, count) + k_list[1] * p_2(gram_3) + k_list[2] * p_1(gram_3)
            for gram_3, count in vocab_l[2].items()}


def cal_linear_perplex(test, k_list, model):
    n = 3
    if k_list[0] == 0:
        n = 2
        if k_list[1] == 0:
            n = 1
    model_n = {gram3[3-n:]: count for gram3, count in model.items()}
    return cal_perplexity(test, n, model_n)


def search_linear_param():
    grams_3 = nltk.ngrams(tokens, 3)
    vocab_3 = nltk.FreqDist(grams_3)

    grams_2 = nltk.ngrams(tokens, 2)
    vocab_2 = nltk.FreqDist(grams_2)

    grams_1 = nltk.ngrams(tokens, 1)
    vocab_1 = nltk.FreqDist(grams_1)

    vocab_l = [vocab_1, vocab_2, vocab_3]
    min = 10000000
    for k_1 in range(0, 6):
        for k_2 in range(0, 6 - k_1):
            k_list = [k_1/5, k_2/5, 1-(k_1 + k_2)/5]
            linear_m = linear_interpolation(k_list, vocab_l)
            perplex_dev = cal_linear_perplex(dev, k_list, linear_m)
            perplex_train = cal_linear_perplex(train, k_list, linear_m)
            print("hyperparameter set:{}, {}, {}:\n"
                  "Perplexity on training set:{}\n"
                  "Perplexity on dev set:{}\n".format(k_list[0], k_list[1], k_list[2], perplex_train, perplex_dev)
                  )
            if perplex_dev < min:
                min = perplex_dev
                best_param = k_list
    perplex_test = cal_linear_perplex(test, k_list, linear_m)
    print("best hyperparameter set:{}, {}, {}\n"
          "Perplexity on test set:{}".format(best_param[0], best_param[1], best_param[2], perplex_test))


search_linear_param()

hyperparameter set:0.0, 0.0, 1.0:
Perplexity on training set:93.99585636227933
Perplexity on dev set:75.43207223220776

hyperparameter set:0.0, 0.2, 0.8:
Perplexity on training set:35.32091550499629
Perplexity on dev set:28.918593160262922

hyperparameter set:0.0, 0.4, 0.6:
Perplexity on training set:26.490784435938586
Perplexity on dev set:23.13301243763304

hyperparameter set:0.0, 0.6, 0.4:
Perplexity on training set:22.00629511068622
Perplexity on dev set:20.027125913975368

hyperparameter set:0.0, 0.8, 0.19999999999999996:
Perplexity on training set:19.197558177553166
Perplexity on dev set:18.05440150900059

hyperparameter set:0.0, 1.0, 0.0:
Perplexity on training set:17.29268572614725
Perplexity on dev set:16.78284067451732

hyperparameter set:0.2, 0.0, 0.8:
Perplexity on training set:11.272936768296475
Perplexity on dev set:13.931227143669025

hyperparameter set:0.2, 0.2, 0.6:
Perplexity on training set:9.795959658306847
Perplexity on dev set:11.87727735869802

hyperparameter set

##### Discussion

best hyperparameter set:1.0, 0.0, 0.0<br/>
Perplexity on training set:4.365120217349759<br/>
Perplexity on dev set:7.297414670618797<br/>
Perplexity on test set:7.296406756886781<br/>

because in this case, vocabulary is build from train set and in the training process zero-probility is not existed. There is no need to use linear interpolation and it is better to use tigram model which is precise.

##### Optional: Optimization

###### Discussion

## 2 Preposition Prediction

In [63]:
!wget -O dev.in https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out

--2022-10-10 23:06:42--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232043 (227K) [text/plain]
Saving to: 'dev.in'

     0K .......... .......... .......... .......... .......... 22% 2.38M 0s
    50K .......... .......... .......... .......... .......... 44% 29.8M 0s
   100K .......... .......... .......... .......... .......... 66% 27.5M 0s
   150K .......... .......... .......... .......... .......... 88% 31.8M 0s
   200K .......... .......... ......                          100% 38.3M=0.03s

2022-10-10 23:06:43 (8.46 MB/s) - 'dev.in' saved [232043/232043]

--2022-10-10 23:06:43--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/

In [66]:
PREP = "<PREP>"


def pre_process2(sentences):
    sentences = add_sentence_tokens(sentences)
    tokens = ' '.join(sentences).split(' ')

    tokens = [token.lower() if token != PREP else token for token in tokens]

    sentences_pred = []
    for i, token in enumerate(tokens):
        if token == PREP:
            sentences_pred.append(tokens[i-1:i+2])
    return sentences_pred


def cal_sentence_perplexity(sentence, prep):
    sentence = [prep if w == PREP else w for w in sentence]
    grams = nltk.ngrams(sentence, 2)
    probs = [bigram_m_2[gram] if gram in bigram_m_2.keys() else math.exp(-100) for gram in grams]
    probs.sort()
    perplex = pow(2, -1*sum(map(math.log, probs)))
    return perplex


def pred_prep(sentence):
    prep_list = ["at", "in", "of", "for", "on"]
    perplex_list = {prep: cal_sentence_perplexity(sentence, prep) for prep in prep_list}
    prep = min(perplex_list.keys(), key=(lambda k: perplex_list[k]))
    return prep


def cal_accuracy():
    actual_res = ' '.join(dev_out).split(' ')
    true_count = 0
    for i, res in enumerate(actual_res):
        if pred_res[i] == res:
            true_count = true_count + 1
    return true_count/len(actual_res)


def output_format():
    format = [line.count(PREP) for line in test_in]
    f = open("3036033675.test.out", "a+")
    start = 0
    for count in format:
        for i in range(0, count):
            if i == count - 1:
                f.write("{}\n".format(test_res[start]))
            else:
                f.write("{} ".format(test_res[start]))
            start = start + 1
    f.close()


bigram_m_2 = build_multi_gram(2)
dev_in = load_data("dev.in")
dev_out = load_data("dev.out")
test_in = load_data("test.in")
sentences = pre_process2(dev_in)
pred_res = [pred_prep(sentence) for sentence in sentences]
accuracy = cal_accuracy()
print(accuracy)

sentences = pre_process2(test_in)
test_res = [pred_prep(sentence) for sentence in sentences]
output_format()

0.7184889211768979


I tried trigram model and bigram model to do preposition prediction, and the result is that, the accuracy of bigram model on dev set is 71.8%, which is a lot larger than that of trigram model. Because the vocabulary and training set are not large encough, useing trigram model will lead to many UNK. Besides, when caculating perplexity, at first, I use UNK to convert grams not seen in traing corpus, to deal with zero-probility. But this method turned out to have low-accuracy. Then I add e-100 to solve zero-probility problem. <br/> 
Theoretically, model with high perplexity on traing will perform better in application. But in this experiment, trigram model with higher perlexity performs worse. Because the model just fit training set which is small.